In [1]:
import os
import datetime
import time

import ipywidgets as widgets

import IPython
import IPython.display
# import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython.display import clear_output

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

from math import sqrt
from numpy import concatenate
import numpy as np
import math

import pandas as pd
from pandas import DataFrame
from pandas import concat

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

# import tensorflow 
# import tensorflow as tf
# from tensorflow import keras

import platform
# print(platform.python_version())
# print(tf.version.VERSION)
# print(np.__version__)



In [ ]:
cwd = os.getcwd()
# cwd

In [ ]:
# File name and Path
cwd = os.getcwd()
path_adress = "/Import_Dataset/"

# ข้อมูล พิกัดสถานที่เเละรายชื้ออ้างอิงสถานที่ตรวจวัด
file_name_st = 'Data_lat_long_Rice research Center'
csv_file_st = cwd + path_adress + file_name_st + '.csv'

df_st = pd.read_csv(csv_file_st)
print("All low RiceCenter {} station" .format(df_st.shape[0]))

data_list_st_num = list(range(1, len(df_st)))
data_list_st_name = []
data_list_st_name = df_st['nameEng'].values.tolist()
data_list_st_lat = df_st['Latitude'].values.tolist()
data_list_st_long = df_st['Longitude'].values.tolist()

In [ ]:
csv_file_st

In [ ]:
st = 10
# st_num = 1
file_name = df_st['nameEng'][data_list_st_num[st]]
name_input = file_name
name_locals = 'stN_' + file_name
print(name_locals)

# File name and Path
csv_file = cwd + path_adress + name_input + '.csv'
# index_col=0 , index_col=None
dataset = pd.read_csv(csv_file, header=0,
                      index_col=0, encoding="TIS-620")
# dataset = dataset.drop(['address'], axis=1)
# dataset = dataset.drop(['year'], axis=1)
# dataset = dataset.drop(['dew'], axis=1)
header_name = list(dataset)
print(header_name)

In [ ]:
# df_temperature = dataset['temp']
df_temperature = pd.DataFrame(dataset['bph'].loc['2015-01-01':'2015-01-31']) 

# df_temperature = 
# the simple moving average over a period of 10 years
# df_temperature['7D-SMA'] = df_temperature.bph.rolling(7, min_periods=1).mean()
# df_temperature['3D-SMA'] = df_temperature.bph.rolling(3, min_periods=1).mean()
df_temperature['7D-SMA'] = df_temperature.bph.rolling(7, min_periods=1).sum()
df_temperature['3D-SMA'] = df_temperature.bph.rolling(3, min_periods=1).sum()
# df_temperature

In [ ]:
# colors for the line plot
colors = ['green', 'red', 'purple']

# line plot - the yearly average air temperature in Barcelona
df_temperature.plot(color=colors, linewidth=3, figsize=(12,6))

# modify ticks size
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend(labels =['Data Input', '10-years SMA', '20-years SMA'], fontsize=14)

# title and labels
plt.title('Data average ', fontsize=18)
plt.xlabel('date-time', fontsize=16)
plt.ylabel('Temperature [°C]', fontsize=16)

In [ ]:
#defining rate
our_rate = 6
 
#apply the rate
constant_subset = df_temperature[::our_rate]
 
#data
constant_subset

In [ ]:
def creat_dataset(st_BPH=0, y_1='2015', y_2='2019', mode=1):
    for i in range(len(st_num)):
        st = i
        file_name = df_st['nameEng'][st_num[st]]
        name_input = file_name
        name_locals = 'stN_' + file_name

        # File name and Path
        csv_file = path_adress + name_input + '.csv'
        # index_col=0 , index_col=None
        dataset = pd.read_csv(csv_file, header=0,
                              index_col=0, encoding="TIS-620")
        dataset = dataset.drop(['address'], axis=1)
        dataset = dataset.drop(['year'], axis=1)
        dataset = dataset.drop(['dew'], axis=1)
        # dataset = dataset.drop(['latitude'], axis=1)
        # dataset = dataset.drop(['longitude'], axis=1)
        dataset.columns
        locals()[name_locals] = dataset
        # print(f'Dataframe name_station: {st+1 , name_input}')
        print('wait......')
        del dataset
        clear_output(wait=True)

    date_start = y_1 + '-01' + '-01'
    date_stop = y_2 + '-12' + '-31'

#---------------------------------------------------------------------------------------------------------------------------------------------------#
    if mode == 0:
        file_name = df_st['nameEng'][st_BPH]
        locals_input = 'stN_' + file_name
        print(locals_input)
        dataset_st = locals()[locals_input].loc[date_start:date_stop]
        frames_st = dataset_st
    else:
        m = 0
        for j in range(len(st_num)):
            # for j in range(df_st.shape[0]):
            # file_name = df_st['nameEng'][j]
            file_name = df_st['nameEng'][st_num[j]]
            locals_input = 'stN_' + file_name
            print(locals_input)
            dataset_st = locals()[locals_input].loc[date_start:date_stop]
            clear_output(wait=True)
            if m == 0:
                frames_st = dataset_st
                m = m+1
                print(m)
            else:
                frames_st = [frames_st, dataset_st]
                frames_st = pd.concat(frames_st)
#---------------------------------------------------------------------------------------------------------------------------------------------------#
    # frames=frames_st[new_colum_2]
    frames = frames_st
    return frames


In [ ]:
frames_train = creat_dataset(1,'2015','2015',0)

In [ ]:
frames_train